# Jump process using Gillespie.jl
Simon Frost (@sdwfrost), 2020-04-27

## Introduction

This implementation of a jump process uses [`Gillespie.jl`](https://github.com/sdwfrost/Gillespie.jl) rather than `DifferentialEquations.jl`.

## Libraries

We will need to install the (unregistered) package, `Gillespie.jl`

In [ ]:
using Pkg
Pkg.add(PackageSpec(url="https://github.com/sdwfrost/Gillespie.jl", rev="master"))

The library can now be loaded along with the other packages.

In [ ]:
using Gillespie
using Random
using StatsPlots
using BenchmarkTools

## Transitions

`Gillespie.jl` expects a single function that returns a vector of all the rates.

In [ ]:
function sir_rates(x,parms)
  (S,I,R) = x
  (β,c,γ) = parms
  N = S+I+R
  infection = β*c*I/N*S
  recovery = γ*I
  [infection,recovery]
end;

The transitions are defined as an array of arrays.

In [ ]:
sir_transitions = [[-1 1 0];[0 -1 1]];

This means that the first rate results in the first variable going down by one, and the second variable going up by one, with the third variable remaining unchanged, etc..


## Time domain

In [ ]:
tmax = 40.0;

## Initial conditions

In [ ]:
u0 = [990,10,0]; # S,I,R

## Parameter values

In [ ]:
p = [0.05,10.0,0.25]; # β,c,γ

## Random number seed

We set a random number seed for reproducibility.

In [ ]:
Random.seed!(1234);

## Running the model

In [ ]:
sol_jump = ssa(u0,sir_rates,sir_transitions,p,tmax);

## Post-processing

`Gillespie.jl` has a convenience function to convert output to a `DataFrame`.

In [ ]:
df_jump = ssa_data(sol_jump);

## Plotting

We can now plot the results.

In [ ]:
@df df_jump plot(:time,
    [:x1 :x2 :x3],
    label=["S" "I" "R"],
    xlabel="Time",
    ylabel="Number")

## Benchmarking

In [ ]:
@benchmark ssa(u0,sir_rates,sir_transitions,p,tmax)